# Python com SQL

- Muitos aplicativos interagem com dados. 
- Algumas linguagens de programação já vem com modulos para usar essas interações.
- Outras linguagens usam uma terceiro pacote, sendo necessario a intalação. 
- Nesta aula vamos mostrar as diferentes bibliotecas de Python SQL.
- Vamos desenvolver uma pequena database para uma aplicação de social media. A database vai consistir de:
    - Usuarios;
    - Posts;
    - Comentarios;;
    - Likes.

### Usando Python SQL para conectar com uma database

- Antes de interagir com qualquer database através de Python, é necessario se conectar com a database. 
- É necessario os servidores dos MySQL e PostgreSQL ligados e rodando antes de executar códigos.

### SQLite

- É bastante utilizado, não é necessario instalação.
- Já vem nas bibliotecas padrões de Python com o nome sqlite3.
- SQLite é serveless e self-contained, pode ler e escrever dados para um arquivo. 


In [41]:
import sqlite3 #importa sqlite3
from sqlite3 import Error #importa a classe Error

def create_connection(path): #Função que indica o caminho para o banco de dados SQLite
    connection = None 
    try:
        connection = sqlite3.connect(path) #Retorna o objeto connection, que é retornado pela função create_connection. Pode ser usado para executar consultas em bancos de dados.
        print("Connection to SQLite DB successful")
    except Error as e:
        print("The error '{e}' occurred")

    return connection    

In [38]:
#Cria uma conexão com o banco de dados SQLite
connection = create_connection("E:\\sm_app.sqlite")

Connection to SQLite DB successful


### Criando Tabelas

- Já criamos o sm_app.
- Vamos criar as tabelas: usuarios, posts, comentarios e likes.
- Vamos criar uma função execute_query(), vai receber o objeto connection e uma string query.
- Contará com um método .execute(), que pode executar qualquer consulta passada na forma de string. 


In [29]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [50]:
#Escrevendo consulta
#Cria uma tabela users com cinco colunas, id, name, age, gender e nationality
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  nationality TEXT
);
"""


#Criando tabela de posts
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""


In [51]:
#Ligará execute_query() para criar a tabela, passando o objeto connection, junto com a create_users_table

execute_query(connection, create_users_table)
execute_query(connection, create_posts_table)  

Query executed successfully
Query executed successfully


In [43]:
#Tabela de comentarios
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""
#tabelas de likes
create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

execute_query(connection, create_comments_table)  
execute_query(connection, create_likes_table) 

Query executed successfully
Query executed successfully


- Criar tabelas em SQLite só é preciso armazenar a consulta em uma variável de sequência e passar essa variável para cursor.execute().

### Inserindo Registros

- Para inserir registros pode ser usado a execute_query(). 

    - Primeiro deve armazenar sua consulta INSERT INTO em uma string.
    - Segundo deve passar o objeto connection e a string query para execute_query().

In [44]:
#Inserindo 5 registros na tabela users:
create_users = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

#Não é necessario especificar o valor da coluna id, a tabela users preencherá automaticamente esses cinco registros 
execute_query(connection, create_users) 

Query executed successfully


In [52]:
#Inserindo seis registros na tabela post:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

execute_query(connection, create_posts)  

Query executed successfully


In [46]:
#Insere comentarios na tabela comments e likes:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, create_comments)
execute_query(connection, create_likes) 

Query executed successfully
Query executed successfully


### Selecionando Registros

- Vamos usar o cursor.execute(), mas depois disso, precisará ligar .fetchall(), método que retorna uma lista de tuplas em que cada tupla é mapeada para a linha correspondente nos registros recuperados. 

In [53]:
#Aceita o objeto connection e a consulta SELECT e retorna o registro selecionado
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print("The error '{e}' occurred")

In [55]:
#Selecionando todos os registros da tabela user
select_users = "SELECT * from users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)
    
#A consulta SELECT seleciona todos os usuários da tabela user, isso é passado para o execute_read_query(), que retorna todos os registros da tabela user.    
#Não é recomendado o uso do SELECT em tabelas grandes, pois pode aumentar o número de operações E/S que aumentam o tráfego da rede.


(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


In [56]:
#Registros da tabela posts:
select_posts = "SELECT * FROM posts"
posts = execute_read_query(connection, select_posts)

for post in posts:
    print(post)

(1, 'Happy', 'I am feeling very happy today', 1)
(2, 'Hot Weather', 'The weather is very hot today', 2)
(3, 'Help', 'I need some help with my work', 2)
(4, 'Great News', 'I am getting married', 1)
(5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(6, 'Party', 'Anyone up for a late-night party today?', 3)


- Você também pode executar consultas complexas envolvendo operações JOIN para recurar dados de duas tabelas relacionadas. 
- É possivel selecionar dados de três tabelas relacionadas implementando vários operadores JOIN.

In [57]:
select_users_posts = """
SELECT
  users.id,
  users.name,
  posts.description
FROM
  posts
  INNER JOIN users ON users.id = posts.user_id
"""

users_posts = execute_read_query(connection, select_users_posts)

for users_post in users_posts:
    print(users_post)

(1, 'James', 'I am feeling very happy today')
(2, 'Leila', 'The weather is very hot today')
(2, 'Leila', 'I need some help with my work')
(1, 'James', 'I am getting married')
(5, 'Elizabeth', 'It was a fantastic game of tennis')
(3, 'Brigitte', 'Anyone up for a late-night party today?')


In [58]:
select_posts_comments_users = """
SELECT
  posts.description as post,
  text as comment,
  name
FROM
  posts
  INNER JOIN comments ON posts.id = comments.post_id
  INNER JOIN users ON users.id = comments.user_id
"""

posts_comments_users = execute_read_query(
    connection, select_posts_comments_users
)

for posts_comments_user in posts_comments_users:
    print(posts_comments_user)

('Anyone up for a late-night party today?', 'Count me in', 'James')
('I need some help with my work', 'What sort of help?', 'Elizabeth')
('I am getting married', 'Congrats buddy', 'Leila')
('It was a fantastic game of tennis', 'I was rooting for Nadal though', 'Mike')
('I need some help with my work', 'Help with your thesis?', 'Leila')
('I am getting married', 'Many congratulations', 'Elizabeth')


In [59]:
cursor = connection.cursor()
cursor.execute(select_posts_comments_users)
cursor.fetchall()

column_names = [description[0] for description in cursor.description]
print(column_names)

['post', 'comment', 'name']


- Where:
    - Executando uma consulta SELECT que retorna a postagem, juntamente com o número total de curtidas que a postagem recebeu.
    - Usando a cláusula WHERE é possivel retornar resultados mais específicos

In [60]:
select_post_likes = """
SELECT
  description as Post,
  COUNT(likes.id) as Likes
FROM
  likes,
  posts
WHERE
  posts.id = likes.post_id
GROUP BY
  likes.post_id
"""

post_likes = execute_read_query(connection, select_post_likes)

for post_like in post_likes:
    print(post_like)

('The weather is very hot today', 1)
('I need some help with my work', 1)
('I am getting married', 2)
('It was a fantastic game of tennis', 1)
('Anyone up for a late-night party today?', 2)


### Atualizando Registros da Tabela

- É bastante simples.
- Faz uso novamente do execute_query().
- Primeiro seleciona a descrição do post:

In [61]:
select_post_description = "SELECT description FROM posts WHERE id = 2"

post_description = execute_read_query(connection, select_post_description)

for description in post_description:
    print(description)

('The weather is very hot today',)


In [62]:
#Depois atualiza a descrição:
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""

execute_query(connection, update_post_description)

Query executed successfully


In [64]:
select_post_description = "SELECT description FROM posts WHERE id = 2"

post_description = execute_read_query(connection, select_post_description)

for description in post_description:
    print(description)
    
    #A saida foi atualizada

('The weather has become pleasant now',)


### Deletando Registros de Tabela

- Usa a consulta DELETE.
- Passar o objeto connection e a string consulta para o registro que deseja excluir.
- Depois execute_query() criará um objeto cursor usando o connection e passará a consulta de string para cursor.execute(), que excluirá os registros.

In [63]:
delete_comment = "DELETE FROM comments WHERE id = 5"
execute_query(connection, delete_comment)

Query executed successfully


In [65]:
posts_comments_users = execute_read_query(
    connection, select_posts_comments_users
)

for posts_comments_user in posts_comments_users:
    print(posts_comments_user)

('Anyone up for a late-night party today?', 'Count me in', 'James')
('I need some help with my work', 'What sort of help?', 'Elizabeth')
('I am getting married', 'Congrats buddy', 'Leila')
('It was a fantastic game of tennis', 'I was rooting for Nadal though', 'Mike')
('I am getting married', 'Many congratulations', 'Elizabeth')
